# Nizami-1.7B CoT Fine-Tune (Unsloth + LoRA)  
> GitHub Actions tarafından otomatik trigger edilir.

In [ ]:
# ========== 1. Environment & Secrets ==========
!pip install python-dotenv -q
import os, dotenv
dotenv.load_dotenv("/kaggle/input/my-secrets/secrets.env")
HF_TOKEN   = os.getenv("HF_TOKEN","").strip()
WANDB_KEY  = os.getenv("WANDB_API_KEY","").strip()

# Kaggle secret’ları notebook’a düşerken otomatik gelir
#assert HF_TOKEN and WANDB_KEY, "HF_TOKEN veya WANDB_API_KEY eksik!"

In [ ]:
# ========== 2. Repo’yu çek ==========
REPO_URL = "https://github.com/Rustamshry34/LLMOps.git"
!git clone $REPO_URL repo
%cd repo
!pip install -r docker/requirements_tra.txt

In [ ]:
# ========== 4. Train (pipeline.txt ile aynı hiper-parametreler) ==========
from unsloth import FastLanguageModel
from datasets import load_dataset
from src.dataset_utils import build_combined_dataset
from trl import SFTTrainer, SFTConfig
import torch
import yaml
from huggingface_hub import login


In [ ]:
login(HF_TOKEN)


In [ ]:
# ------ YAML’ı oku ------
with open("configs/training_config.yaml", "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-0.6B"
    max_seq_length=cfg["max_length"],
    dtype=torch.float16,
    load_in_4bit=False,
    device_map={"": 0}
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing=False
)


In [ ]:
# ----- data -----
combined_ds = build_combined_dataset(tokenizer, non_reasoning_pct=0.0, seed=3407)


In [ ]:
# ----- training -----
from trl import SFTTrainer, SFTConfig
import wandb
wandb.login(key=WANDB_KEY)


In [ ]:
args = SFTConfig(**cfg)

text = "text"
trainer = SFTTrainer(model=model, tokenizer=tokenizer, args=args, train_dataset=combined_ds, dataset_text_field=text)


In [ ]:
trainer.train()


In [ ]:
model.save_pretrained(cfg["output_dir"])
tokenizer.save_pretrained(cfg["output_dir"])
wandb.finish()

In [ ]:
# ========== 6. Kaggle çıktısını GitHub’a bildir (status = complete) ==========
print("✅ Kaggle eğitimi tamamlandı.")